<a href="https://colab.research.google.com/github/bahrad/Covid/blob/main/Covid_Predict_Omicron_Resistance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Initialization

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

import numpy as np
import os
import csv

import pandas as pd
import pickle

In [2]:
from google.colab import drive, files
# drive.mount('/content/drive')

FILELOC = "/content/drive/My Drive/COVID_Python/"

In [3]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.TPUStrategy(tpu)
    tpu_env=True
except ValueError:
    print('Not connected to a TPU runtime.')
    tpu_env=False

Running on TPU  ['10.99.211.42:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.99.211.42:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.99.211.42:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


##Define models

In [4]:
def reset_model(regress, singleclass, multiclass, output_multiheadatt, use_att, nclasses=4):

    if output_multiheadatt:
        model_fn = AttMod_2
    else:
        model_fn = AttModel
    model = model_fn(L=ismlen,
                     vocab_size=len(aa_list)+1,
                     embdim = ENCDIM,
                     numheads = NHEADS,
                     ffdim = FFDIM,
                     num_dense = NDENSE,
                     mask_zero=True,
                     dropout_rate = DROPRATE,
                     trans_drop = TRANSDROPRATE,
                     Nt = NT,
                     W = 1, Nc = NC, Nl = NL,
                     regress=regress, singleclass=singleclass,
                     multiclass=multiclass, use_att=use_att,
                     nclasses=nclasses,
                     )
    
    optimizer = keras.optimizers.Adam(learning_rate=LEARN_RATE)
    if regress:
        loss = keras.losses.MeanSquaredError()
        metrics = [keras.metrics.MeanSquaredError(name='mse'),
            keras.metrics.MeanSquaredLogarithmicError(name='msle'),
            keras.losses.MeanAbsoluteError(name='mae')
            ]
    if singleclass:
        loss = keras.losses.BinaryCrossentropy()
        metrics = [keras.metrics.BinaryAccuracy(name='acc')]
    if multiclass:    
        loss = keras.losses.SparseCategoricalCrossentropy()
        metrics = [keras.metrics.SparseCategoricalAccuracy(name='acc')]
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics,)
                #   steps_per_execution = STEPS_PER_EXECUTION,)
    
    return model

In [5]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim, mask_zero=False):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size,
                                                output_dim=embed_dim,
                                                mask_zero=mask_zero)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim,
                                              mask_zero=mask_zero)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

def linear01(x):
    return tf.clip_by_value(x, clip_value_min=0, clip_value_max=1)

In [6]:
def AttMod_2(L, vocab_size, embdim, numheads, ffdim, num_dense=False,
             mask_zero=False, dropout_rate=False, trans_drop=0.1,
             Nt=1, W=False, Nc=False, Nl=False,
             regress=True, singleclass=False, multiclass=False, use_att=True,
             nclasses=4):

    inpTensor = keras.Input(shape=(L,))
    x = inpTensor

    if mask_zero:
        x = keras.layers.Masking(mask_value=0)(x)   

    x = TokenAndPositionEmbedding(L, vocab_size, embdim, mask_zero)(x)

    if W and Nc and Nl:
        for n in range(Nl):
            x = keras.layers.Conv1D(filters = Nc,
                                kernel_size = W,
                                activation = 'relu',
                                padding = 'same',
                                )(x)
            if n > 1 and n < Nl-1:
                x = keras.layers.BatchNormalization()(x)

    y, attout = keras.layers.MultiHeadAttention(num_heads=numheads, key_dim=embdim,
                                                )(x, x, return_attention_scores=True)
    y = keras.layers.Dropout(trans_drop)(y)
    z = keras.layers.LayerNormalization(epsilon=1e-6)(x + y)
    z1 = keras.Sequential( [keras.layers.Dense(ffdim, activation="relu"), keras.layers.Dense(embdim),])
    z1 = keras.layers.Dropout(trans_drop)(z)
    x = keras.layers.LayerNormalization(epsilon=1e-6)(z + z1)

    if use_att:
        # Attention layer
        h = keras.layers.TimeDistributed(keras.layers.Dense(Nc, activation='tanh'))(x)
        attention = keras.layers.TimeDistributed(keras.layers.Dense(1, activation='tanh'))(h)
        attention = keras.layers.Flatten()(attention)  
        attention = keras.layers.Softmax(axis=1, name='attention')(attention) # normalize attention values
        attention = keras.layers.RepeatVector(Nc)(attention)
        attention = keras.layers.Permute([2, 1])(attention)
        representation = keras.layers.multiply([h, attention])
        representation = tf.math.reduce_sum(representation, axis = 1)
        x = representation
    else:
        x = keras.layers.GlobalAveragePooling1D()(x)

    if num_dense:
        x = keras.layers.Dense(num_dense, activation = 'relu')(x)
    if dropout_rate:
        x = keras.layers.Dropout(Params[dropout_rate])(x)

    if regress:
        finalOut = keras.layers.Dense(1, activation=linear01)(x)
    if singleclass:
        finalOut = keras.layers.Dense(1, activation='sigmoid')(x)
    if multiclass:
        finalOut = keras.layers.Dense(nclasses, activation='softmax')(x)

    # define the model's start and end points    
    model = keras.Model(inpTensor,finalOut)

    return model

In [50]:
def AttModel(L, vocab_size, embdim, numheads, ffdim, num_dense=False,
             mask_zero=False, dropout_rate=False, trans_drop=0.1,
             Nt=1, W=False, Nc=False, Nl=False,
             regress=True, singleclass=False, multiclass=False, use_att=True,
             nclasses=4):

    inpTensor = keras.Input(shape=(L,))
    x = inpTensor

    if mask_zero:
        x = keras.layers.Masking(mask_value=0)(x)   

    x = TokenAndPositionEmbedding(L, vocab_size, embdim, mask_zero)(x)

    if W and Nc and Nl:
        for n in range(Nl):
            x = keras.layers.Conv1D(filters = Nc,
                                kernel_size = W,
                                activation = 'relu',
                                padding = 'same',
                                )(x)
            if n > 1 and n < Nl-1:
                x = keras.layers.BatchNormalization()(x)

    for n in range(Nt):
        x = TransformerBlock(Nc, numheads, ffdim, rate=trans_drop)(x)

    if use_att:
        # Attention layer
        h = keras.layers.TimeDistributed(keras.layers.Dense(Nc, activation='tanh'))(x)
        attention = keras.layers.TimeDistributed(keras.layers.Dense(1, activation='tanh'))(h)
        attention = keras.layers.Flatten()(attention)  
        attention = keras.layers.Softmax(axis=1, name='attention')(attention) # normalize attention values
        attention = keras.layers.RepeatVector(Nc)(attention)
        attention = keras.layers.Permute([2, 1])(attention)
        representation = keras.layers.multiply([h, attention])
        representation = tf.math.reduce_sum(representation, axis = 1)
        x = representation
    else:
        x = keras.layers.GlobalAveragePooling1D()(x)

    if num_dense:
        x = keras.layers.Dense(num_dense, activation = 'relu')(x)
    if dropout_rate:
        x = keras.layers.Dropout(Params[dropout_rate])(x)

    if regress:
        # finalOut = keras.layers.Dense(1, activation=linear01)(x)
        finalOut = keras.layers.Dense(1, activation='sigmoid')(x)       
    if singleclass:
        finalOut = keras.layers.Dense(1, activation='sigmoid')(x)
    if multiclass:
        finalOut = keras.layers.Dense(nclasses, activation='softmax')(x)

    # define the model's start and end points    
    model = keras.Model(inpTensor,finalOut)

    return model

In [41]:
ismlen = 1273

def tokenize_sequences(data_dataframe, SeqCol='ISM', seqlen=1273):
    def f(x):
        if len(x) < seqlen:
            return x + '*'*(seqlen-len(x))
        elif len(x) > seqlen:
            return x[:seqlen]
        else:
            return x
    data = np.vstack(data_dataframe[SeqCol].apply(f).apply(lambda x: np.array(list(x))))
    aa_list = ['A', 'R', 'N', 'D', 'C', 'Q', 'E',
            'G', 'H', 'I', 'L', 'K', 'M', 'F',
            'P', 'S', 'T', 'W', 'Y', 'V', '-',
            ]
    aa_tokenizer = {aa_list[k]:k+1 for k in range(len(aa_list))}
    aa_tokenizer['*'] = 0
    aa_tokenizer['X'] = 0
    # optionally handle B, J, Z ambiguities
    # Asx	B	Aspartic acid or Asparagine (D or N)
    # Glx	Z	Glutamic acid or Glutamine (E or Q)
    # Xaa	X	Any amino acid
    # Xle	J	Leucine or Isoleucine (L or I)
    aa_tokenizer['B'] = 0
    aa_tokenizer['Z'] = 0
    aa_tokenizer['J'] = 0

    return np.vectorize(aa_tokenizer.get)(data)

aa_list = ['A', 'R', 'N', 'D', 'C', 'Q', 'E',
        'G', 'H', 'I', 'L', 'K', 'M', 'F',
        'P', 'S', 'T', 'W', 'Y', 'V', '-',
        ]

In [8]:
# These parameters are currently hard-coded
ENCDIM = 1500
NC = 300
NL = 1
NT = 1
NHEADS = 8
FFDIM = 64
NDENSE = 64
TRANSDROPRATE = 0.4
DROPRATE = 0

LEARN_RATE = 0.0001

BATCH_SIZE = 48

STEPS_PER_EXECUTION = 50

##Prepare sequences for training / validation

Based on protein sequence data downloaded from GISAID (http://www.gisaid.org) that was current as of October 1, 2021.

Aligned sequences (aligned using BLOSUM62 with local pairwise SSW method implemented in scikit-bio)

In [9]:
# mdf = pd.read_csv(FILELOC+'metadata.tsv', '\t')
# seqs = pd.read_csv(FILELOC+'align1001_all.csv')
# df = pd.merge(left=mdf[['Accession ID', 'Collection date', 'Clade', 'Pango lineage', 'Location']],
#               right=seqs, left_on='Accession ID', right_on="SequenceID", how='inner')
# df.rename(columns={'Accession ID':'SequenceID', 'Pango lineage':'Lineage'}, inplace=True)
# df = df.loc[:,~df.columns.duplicated()]
# df['Country'] = df['Location'].apply(lambda x: x.split('/')[1].strip())
# from dateutil.parser import parse as dateparse
# df = df[['SequenceID', 'Date', 'Clade', 'Lineage', 'Country', 'MaskedSeq', 'Collection date', 'Location']]
# # with open(FILELOC + 'Spike1001_all.pkl', 'wb') as f:
# #     pickle.dump(df, f)
# with open(FILELOC + 'Spike1001_all.pkl', 'rb') as f:
#     df = pickle.load(f)
# df = df[df.Date > 0]
# df_reldate = df.groupby("MaskedSeq")["Date"].apply(list).to_frame()
# df_reldate['First_Date'] = df_reldate.Date.apply(np.amin)
# df_reldate['Last_Date'] = df_reldate.Date.apply(np.amax)
# df_reldate['Peak_Date'] = df_reldate.Date.apply(lambda x: np.median(np.argwhere(np.bincount(x)==np.amax(np.bincount(x)))))
# df_reldate['Count'] = df_reldate.Date.apply(len)
# max_first_date = max(df_reldate.First_Date)
# df_reldate['relfirstdate'] = df_reldate['First_Date'].apply(lambda x: x/max_first_date)
# with open(FILELOC + 'Spike1001_allseqs_reldate.pkl', 'wb') as f:
#     pickle.dump(df_reldate, f)
# ddf = df.join(df_reldate, on='MaskedSeq', lsuffix='', rsuffix='_List')
# with open(FILELOC + 'Spike1001_allseqs_reldate_metadata.pkl', 'wb') as f:
#     pickle.dump(ddf, f)
# with open(FILELOC + 'Spike1001_allseqs_reldate.pkl', 'rb') as f:
#     alignseqs = pickle.load(f)
# with open(FILELOC + 'Spike1001_allseqs_reldate_metadata.pkl', 'rb') as f:
#     aligndf = pickle.load(f)
# print(len(aligndf))
# aligndf.drop(aligndf[aligndf.Lineage.isna()].index, inplace=True)
# print(len(aligndf))
# adf = aligndf.groupby('MaskedSeq')['Lineage'].apply(lambda x: x.value_counts().index[0]).to_frame()
# adfm = adf.join(alignseqs, how='left')
# with open(FILELOC + 'aligned_sequence_lineages_1001.pkl', 'wb') as f:
#     pickle.dump([adf, adfm], f)

Raw sequence data

In [10]:
# df = pd.read_csv(FILELOC + 'results1001_Spike.csv')
# df.drop(columns='index', inplace=True)
# mdf = pd.read_csv(FILELOC+'metadata.tsv', '\t')
# mdf = pd.merge(left=mdf[['Accession ID', 'Collection date', 'Clade', 'Pango lineage', 'Location']],
#               right=df, left_on='Accession ID', right_on="SequenceID", how='inner')
# mdf.rename(columns={'Pango lineage':'Lineage'}, inplace=True)
# print(len(mdf))
# mdf['Country'] = mdf['Location'].apply(lambda x: x.split('/')[1].strip())
# mdf['Country'].fillna('Unknown', inplace=True)
# REFDATE = '2019-12-01'; refdt = dateparse(REFDATE)
# def f(x):
#     return (dateparse(x) - refdt).days
# mdf['reldate'] = mdf['Collection date'].apply(f)
# # with open('RawSequence_Metadata_1001.pkl', 'wb') as f:
# #     pickle.dump(mdf, f)
# # mdf.to_csv('RawSequence_Metadata_1001.csv', index=False)
# # !tar -cvzf RawSequence_Metadata_1001.csv.tar.gz RawSequence_Metadata_1001.csv
# df_reldate = mdf.groupby("Spike")["reldate"].apply(list).to_frame()
# df_reldate.rename(columns={"reldate":"Date"}, inplace=True)
# df_reldate['First_Date'] = df_reldate.Date.apply(np.amin)
# df_reldate['Last_Date'] = df_reldate.Date.apply(np.amax)
# df_reldate['Count'] = df_reldate.Date.apply(len)
# max_first_date = max(df_reldate.First_Date)
# df_reldate['relfirstdate'] = df_reldate['First_Date'].apply(lambda x: x/max_first_date)
# df_reldate.reset_index().to_csv(FILELOC + 'RawSequence1001_RelDate.csv', index=False)
# rawseqs = pd.read_csv(FILELOC + 'RawSequence1001_RelDate.csv')
# with open(FILELOC + 'RawSequence_Metadata_1001.pkl', 'rb') as f:
#     rawdf = pickle.load(f)
# print(len(rawdf))
# rawdf.drop(rawdf[rawdf.Lineage.isna()].index, inplace=True)
# print(len(rawdf))
# rdf = rawdf.groupby('Spike')['Lineage'].apply(lambda x: x.value_counts().index[0]).to_frame()
# rsdf = rawseqs.set_index('Spike', drop=True)
# rdfm = rdf.join(rsdf, how='left')
# with open(FILELOC + 'raw_sequence_lineages_1001.pkl', 'wb') as f:
#     pickle.dump([rdf, rdfm], f)

##Get Omicron spike sequences

In [ ]:
# !pip install scikit-bio
# import skbio
# from skbio import TabularMSA, DNA, Protein
# from skbio.alignment import local_pairwise_align_ssw
# !pip install biopython
# from Bio.Align import substitution_matrices
# import itertools

In [132]:
DIR = FILELOC + 'coronavirus_spike/'
SPIKEFILE = DIR + 'EPI_ISL_402124-S.fasta'
# FASTAFILE = DIR + 'gisaid_hcov-19_2021_11_28_08.fasta'
OMICRONFILE = DIR + 'gisaid_hcov-19_2021_12_01_22.fasta'

seqs = [seq for seq in skbio.io.read(OMICRONFILE, format='fasta')]
spike = DNA.read(SPIKEFILE)  # reference sequence from NCBI
align = [local_pairwise_align_ssw(DNA(seq),spike) for seq in seqs]
# align is a triplet of (alignment, score, start_end_positions)
spike_prot = ''.join([x.decode() for x in spike.translate().values])

# # Two ways of identifying aligned sequences (generate the same results)

# find aligned spike sequences without degenerates to translate
no_degenerates = [k for k in range(len(seqs)) if not align[k][0][0].has_degenerates()]
omicron_protein_seqs = [align[k][0][0].degap().translate() for k in no_degenerates]
omicron_seqs = [''.join([x.decode() for x in seq.values]) for seq in omicron_protein_seqs]
omicron_metadata = [seq.metadata['id'] for seq in omicron_protein_seqs]

# # find aligned spike sequences without degenerates to translate
# no_degenerates = [k for k in range(len(seqs)) if not align[k][0][0].has_degenerates()]
# spikelocs = [align[k][2][0] for k in range(len(seqs))]
# omicron_spike_seqs = [seqs[k][spikelocs[k][0]:spikelocs[k][1]+1] for k in no_degenerates]
# omicron_protein_seqs = [DNA(seq).translate() for seq in omicron_spike_seqs]
# omicron_seqs = [''.join([x.decode() for x in seq.values]) for seq in omicron_protein_seqs]
# omicron_metadata = [seq.metadata['id'] for seq in omicron_protein_seqs]

omicrondf = pd.DataFrame.from_dict({'Metadata':omicron_metadata})
omicrondf['SeqID'] = omicrondf.Metadata.apply(lambda x:x.split('|')[1])
omicrondf['Raw'] = omicron_seqs
# omicrondf['seqlen'] = omicrondf['Spike'].apply(len)

# # generate modified aligned sequences where there are insertions relative to
# # reference sequence, i.e., where the inserted residues are deleted

blosum62 = substitution_matrices.load("BLOSUM62")
def f(x):
    alignment,score,startend = local_pairwise_align_ssw(Protein(x),
                                                        Protein(spike_prot),
                                                        substitution_matrix=blosum62)
    seqalign = ''.join([x.decode() for x in alignment[0].values])
    refalign = ''.join([x.decode() for x in alignment[1].values])
    return (seqalign, refalign)

alignments = omicrondf['Raw'].apply(f).tolist()
omicrondf['seqalign'] = [a[0] for a in alignments]
omicrondf['refalign'] = [a[1] for a in alignments]

def f(x):
    seqalign, refalign = x
    try:
        if '-' in seqalign: # gap in aligned reference sequence indicates insertion
            gaps = [0 if c == '-' else 1 for c in refalign]
            return ('').join(itertools.compress(seqalign, gaps))
        else:
            return seqalign
    except:
        print('Error')
        return np.nan

# also remove stopcodons
MAXLEN = 1273
def remstop(x):
    if x[-1] == '*' and len(x) > MAXLEN:
        return x[:-1]
    else:
        return x
omicrondf['Align'] = omicrondf[['seqalign','refalign']].apply(f, axis=1).apply(remstop)

omicrondf.to_csv(FILELOC + 'omicron_spike_20211201.csv',index=False)

#Predict vaccine resistance

Cromer D, Steain M, Reynaldi A, Schlub TE, Wheatley AK, Juno JA, Kent SJ, Triccas JA, Khoury DS, Davenport MP. Neutralising antibody titres as predictors of protection against SARS-CoV-2 variants and the impact of boosting: a meta-analysis. Lancet Microbe. 2021 Nov 15. doi: 10.1016/S2666-5247(21)00267-6. Epub ahead of print. PMID: 34806056; PMCID: PMC8592563.

https://pubmed.ncbi.nlm.nih.gov/34806056/

Kappa (B.1.617.1) has 6.8x less susceptible, while Delta (B.1.617.2) is 2.9x. Edara VV, Lai L, Sahoo MK, Floyd K, Sibai M, Solis D, Flowers MW, et al. Infection and vaccine-induced neutralizing antibody responses to the SARS-CoV-2 B.1.617.1 variant. bioRxiv [Preprint]. 2021 May 10:2021.05.09.443299. doi: 10.1101/2021.05.09.443299. Update in: N Engl J Med. 2021 Jul 7;: PMID: 34013272; PMCID: PMC8132229. https://www.nejm.org/doi/full/10.1056/NEJMc2107799

Potentially relevant to AY.4:
Arora, P., Kempf, A., Nehlmeier, I. et al. Delta variant (B.1.617.2) sublineages do not show increased neutralization resistance. Cell Mol Immunol 18, 2557–2559 (2021). https://doi.org/10.1038/s41423-021-00772-y

Uriu K, Kimura I, Shirakawa K, Takaori-Kondo A, Nakada TA, Kaneda A, Nakagawa S, Sato K; Genotype to Phenotype Japan (G2P-Japan) Consortium. Neutralization of the SARS-CoV-2 Mu Variant by Convalescent and Vaccine Serum. N Engl J Med. 2021 Nov 3:NEJMc2114706. doi: 10.1056/NEJMc2114706. Epub ahead of print. PMID: 34731554; PMCID: PMC8609602. https://www.nejm.org/doi/full/10.1056/NEJMc2114706

"Virus neutralization assays, performed with the use of serum samples obtained from 13 persons who had recovered from Covid-19 who were infected early in the pandemic (April through September 2020), showed that the mu variant was 10.6 times as resistant to neutralization as the B.1 lineage virus (parental virus), which bears the D614G mutation (Figure 1B). Assays performed with serum samples obtained from 14 persons who had received the BNT162b2 vaccine showed that the mu variant was 9.1 as resistant as the parental virus (Figure 1C). Although the beta variant (a variant of concern) was thought to be the most resistant variant to date,3,4 the mu variant was 2.0 as resistant to neutralization by convalescent serum (Figure 1B) and 1.5 times as resistant to neutralization by vaccine serum as the beta variant (Figure 1C)."
For below: Parental, Alpha, Beta, Gamma, Delta, Epsilon (B.1.429, B.1.427), Lambda, Mu
Convalescent: -, 2.3, 7.0, 2.9, 3.8, 2.1, 3.4, 10.6x reduction
Vaccine: -, 2.3, 7.6, 2.9, 2.9, 3.0, 1.9, 9.1

For Lambda:
Infectivity and immune escape of the new SARS-CoV-2 variant of interest Lambda
Mónica L. Acevedo, Luis Alonso-Palomares, Andrés Bustamante, Aldo Gaggero, Fabio Paredes, Claudia P. Cortés, Fernando Valiente-Echeverría, Ricardo Soto-Rifo
medRxiv 2021.06.28.21259673; doi: https://doi.org/10.1101/2021.06.28.21259673

"We observed an increased infectivity mediated by the Lambda spike protein that was even higher than that of the D614G (lineage B) or the Alpha and Gamma variants. Compared to the Wild type (lineage A), neutralization was decreased by 3.05-fold for the Lambda variant while it was 2.33-fold for the Gamma variant and 2.03-fold for the Alpha variant."

Delta variants

Baoling Ying, Bradley Whitener, Laura A. VanBlargan, Ahmed O. Hassan, Swathi Shrihari, Chieh-Yu Liang, Courtney E. Karl, Samantha Mackin, Rita E. Chen, Natasha M. Kafai, Samuel H. Wilks, Derek J. Smith, Juan Manuel Carreño, Gagandeep Singh, Florian Krammer, Andrea Carfi, Sayda M. Elbashir, Darin K. Edwards, Larissa B. Thackray, Michael S. Diamond,  Protective activity of mRNA vaccines against ancestral and variant SARS-CoV-2 strains, Science Translational Medicine, 0, 0, (undefined). /doi/10.1126/scitranslmed.abm3302

"Geometric mean neutralizing Ab titers (GMTs) for the Pfizer/BioNtech BNT162b2-elicited plasma were respectively reduced 3.0-, 2.4-, and 4.1-fold for B.1.617.1, B.1.617.2, and B.1.617.2+ S (GMTs 86, 110, and 63), compared to G614 S (GMT: 260) (Fig. 1A and figs. S1 and S2). Moderna mRNA-1273-elicited plasma GMTs were reduced 4.1-, 2.6-, and 9.5-fold for B.1.617.1, B.1.617.2, and B.1.617.2+ S (GMTs 170, 270, and 74), respectively, compared to G614 S (GMT: 700) (Fig. 1B and figs. S1 and S2). The average neutralization potency of the Janssen Ad26.COV2.S-elicited plasma was reduced 3.0-, 2.4-, and 3.5-fold for B.1.617.1, B.1.617.2, and B.1.617.2+ S (GMTs 10, 13, and 8.8), respectively, compared to G614 S (GMT: 31)"

In [113]:
### Lineage -> Impact on neutralization mapping

# Cromer D, Steain M, et al. + Edara VV, Lai L, et al. for Kappa (based on Cromer) +
# Uriu et al. for B lineages (D614G) and Mu (B.1.621), Epsilon (B.1.427/429)
lins = ['A', 'A.1', 'A.2.5',
        'B', 'B.1', 'B.1.1.7',
        'B.1.351', 'B.1.351.2', 'B.1.351.5',
        'P.1', 'B.1.617.2', 'B.1.617.1',
        'B.1.621', 'B.1.427', 'B.1.429',
        'C.37']
neutr = [1, 1, 1,
         1/1.17, 1/1.17, 1/1.6,
         1/8.8, 1/8.8, 1/8.8,
         1/3.5, 1/3.9, 1/6.5,
         1/10.5, 1/3.0, 1/3.0,
         1/4.6]

# Uriu et al. for all lineages (convalescent response) 
# Parental, Alpha, Beta, Gamma, Delta, Epsilon (B.1.429, B.1.427), Lambda, Mu Convalescent

lins = ['B.1', 'B.1.1.7', 'B.1.351', 'P.1', 'B.1.617.2', 'B.1.427', 'B.1.429',
        'C.37', 'C.37.1', 'C39', 'C40', 'B.1.621']
neutr = 1 / np.array([1, 2.3, 7.0, 2.9, 3.8, 2.1, 2.1,
                      3.4, 3.4, 3.4, 3.4, 10.6])

# Uriu et al. and Ying et al.
lins = ['B.1', 'B.1.1.7', 'B.1.351', 'P.1', 'B.1.617.2', 'B.1.427', 'B.1.429',
        'C.37', 'B.1.621', 'B.1.617.1', 'B.1.617.2+']
neutr = 1 / np.array([1, 2.3, 8.8, 2.9, 3.8, 2.1, 2.1,
                      3.4, 13.3, 5.35, 9])


assert len(lins) == len(neutr)
numlins = len(lins)

labelmap = {lins[k]:neutr[k] for k in range(numlins)}

In [114]:
with open(FILELOC + 'aligned_sequence_lineages_1001.pkl', 'rb') as f:
    _, tdf = pickle.load(f)
tdf.reset_index(drop=False, inplace=True)

# with open(FILELOC + 'raw_sequence_lineages_1001.pkl', 'rb') as f:
#     _, tdf = pickle.load(f)
# datadf.reset_index(drop=False, inplace=True)

In [115]:
tdf['Lineage_Orig'] = tdf['Lineage']
def f(x):
    if x[483] != "E":
        return 'B.1.617.2+'
    else:
        return x
tdf.loc[tdf[tdf.Lineage_Orig=='B.1.617.2'].index, 'Lineage'] = tdf.loc[tdf.Lineage_Orig=='B.1.617.2']['MaskedSeq'].apply(f)

In [116]:
tdf = tdf[tdf.Lineage.isin(lins)]
print([len(tdf[tdf.Count >= c]) for c in range(1,6)])
# filter sequences by removing any with 'X' or '*' characters
tdf = tdf[~tdf.MaskedSeq.str.contains('X|\*')]
print([len(tdf[tdf.Count >= c]) for c in range(1,6)])

[220603, 30331, 17778, 12938, 10210]
[28659, 12006, 8103, 6277, 5176]


In [117]:
MINCOUNT = 1
tdf[tdf.Count >= MINCOUNT]['Lineage'].map(labelmap).value_counts()

0.434783    18816
0.344828     3052
1.000000     2568
0.476190     1672
0.113636      865
0.294118      730
0.186916      442
0.075188      413
0.111111      101
Name: Lineage, dtype: int64

In [118]:
traindf = tdf[tdf.Count >= MINCOUNT].copy()
traindf['label'] = traindf.Lineage.map(labelmap)

In [120]:
MAX_SAMPLE = 200

# from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours
# SRS = 2077
# sm = RandomUnderSampler(sampling_strategy='majoriy', random_state=SRS, replacement=False)
# xt, yt = sm.fit_resample(xtrain, traindf.Lineage.map({lins[ind]:ind for ind in range(len(lins))}))

### Manual undersampling
# dropindices = {}
# for label in np.unique(traindf.label):
#     n = len(traindf[traindf.label==label])
#     if n > MAX_SAMPLE:
#         print(1/label, len(traindf))
#         dropindices[label] = np.random.choice(traindf[traindf.label==label].index, n-MAX_SAMPLE, replace=False)
#         traindf.drop(dropindices[label], inplace=True)
#         print(len(traindf))

## Take the top samples
dropindices = {}
for label in np.unique(traindf.label):
    n = len(traindf[traindf.label==label])
    if n > MAX_SAMPLE:
        print(1/label, len(traindf))
        dropindices[label] = traindf[traindf.label==label].sort_values(by='Count',
                                                                       ascending=False).index[MAX_SAMPLE:]   
        traindf.drop(dropindices[label], inplace=True)
        print(len(traindf))

13.3 28659
28446
8.8 28446
27781
5.35 27781
27539
3.4 27539
27009
2.9 27009
24157
2.3 24157
5541
2.1 5541
4069
1.0 4069
1701


In [122]:
xtrain = tokenize_sequences(traindf, 'MaskedSeq', ismlen)
# xtrain = tokenize_sequences(traindf, 'Spike', ismlen)

ytrain = traindf.Lineage.map(labelmap).values

In [123]:
NUM_EPOCHS = 200
PATIENCE = 50   # 10

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'loss',
    verbose = 1,
    patience = PATIENCE,
    mode = 'auto',
    min_delta = 0,
    baseline = 0.2,
    restore_best_weights = True
    )

In [ ]:
n_epochs = 0
while (n_epochs < (PATIENCE + 5)):
    tf.keras.backend.clear_session()
    with tpu_strategy.scope():
        model = reset_model(regress=True, singleclass=False, multiclass=False,
                            output_multiheadatt=False, use_att=True)
    history = model.fit(xtrain, ytrain,
                        batch_size = BATCH_SIZE,
                        epochs = NUM_EPOCHS,
                        verbose = 2,
                        callbacks = [early_stopping],
                        )
    n_epochs = len(history.history['loss'])

In [ ]:
WEIGHTFILE = "coronavirus_vaxresist_20211201.h5"

# model.save_weights(FILELOC + WEIGHTFILE, save_format='h5', overwrite=True)
# with open(FILELOC + WEIGHTFILE + '_setup.pkl','wb') as f:
#     pickle.dump([tdf, traindf, dropindices, xtrain, ytrain, MINCOUNT, MAX_SAMPLE, labelmap], f)

In [20]:
with tpu_strategy.scope():
    model = reset_model(regress=True, singleclass=False, multiclass=False,
                        output_multiheadatt=True, use_att=True)
    model.load_weights(FILELOC + WEIGHTFILE)
    model.compile()
with open(FILELOC + WEIGHTFILE + '_setup.pkl','rb') as f:
    tdf, traindf, dropindices, xtrain, ytrain, MINCOUNT, MAX_SAMPLE, labelmap = pickle.load(f)

In [ ]:
from sklearn.metrics import mean_squared_error
from collections import Counter

In [141]:
# omicron = pd.read_csv(FILELOC + "omicron_spike_20211128.csv")
omicron = pd.read_csv(FILELOC + "omicron_spike_20211201.csv")
omicron['seqlen'] = omicron.Align.apply(len)
omicron.drop(omicron[omicron.seqlen<1273].index, inplace=True)
omicron.reset_index(inplace=True, drop=True)
otok = tokenize_sequences(omicron,'Align',ismlen)

In [142]:
omicron['Predict'] = model.predict(otok)

In [143]:
np.mean(omicron.Predict)

0.10882622003555298

In [144]:
np.max(omicron.Predict)

0.1921105980873108

In [145]:
np.min(omicron.Predict)

0.05763649940490723

In [147]:
Counter(omicron.Predict)

Counter({0.05763649940490723: 12,
         0.11000391840934753: 1,
         0.11113205552101135: 1,
         0.1113305389881134: 2,
         0.11178094148635864: 87,
         0.17104101181030273: 1,
         0.17186003923416138: 2,
         0.1859753429889679: 1,
         0.1921105980873108: 1})